# **Faster RCNN in PyTorch**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##### Faster RCNN has replaced the selective search with RPN (Region proposal network) network.

In [ ]:
import torch
image = torch.zeros((1, 3, 800, 800)).float()
#bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]], dtype=torch.float) # [y1, x1, y2, x2] format
bbox = torch.tensor([[20, 30, 400, 500], [300, 400, 500, 600]], dtype=torch.float)
labels = torch.LongTensor([6, 8]) # 0 represents background
sub_sample = 16

In [ ]:
import torchvision
dummy_img = torch.zeros((1, 3, 800, 800)).float()
print(dummy_img)
#Out: torch.Size([1, 3, 800, 800])

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])


In [ ]:
# Listing all the layers of the VGG16.
model = torchvision.models.vgg16(pretrained=True)
fe = list(model.features)
print(fe)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [ ]:
req_features = []
k = dummy_img.clone()
for i in fe:
    k = i(k)
    if k.size()[2] < 800//16:
        break
    req_features.append(i)
    out_channels = k.size()[1]
print(len(req_features)) #30
print(out_channels) # 512

30
512


In [ ]:

#Convert this list into a Sequential module.
from torch import nn
faster_rcnn_fe_extractor = nn.Sequential(*req_features)

In [ ]:
sample_output = faster_rcnn_fe_extractor(dummy_img)
print(sample_output.shape)

torch.Size([1, 512, 50, 50])


In [ ]:
faster_rcnn_fe_extractor

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_si

In [ ]:
# Now this faster_rcnn_fe_extractor can be used as our backend. Lets compute the features
out_map = faster_rcnn_fe_extractor(image)
print(out_map.size())
#Out: torch.Size([1, 512, 50, 50])

torch.Size([1, 512, 50, 50])


At each pixel location on the feature map, We need to generate 9 anchor boxes (number of anchor_scales and number of ratios) and each anchor box will have ‘y1’, ‘x1’, ‘y2’, ‘x2’. So at each location anchor will have a shape of (9, 4). Lets begin with a an empty array filled with zero values.

In [ ]:
import numpy as np
ratio = [0.5, 1, 2]
anchor_scales = [8, 16, 32]

anchor_base = np.zeros((len(ratio) * len(anchor_scales), 4), dtype=np.float32)
print(anchor_base)

ctr_y = sub_sample / 2.
ctr_x = sub_sample / 2.

print(ctr_y, ctr_x)

for i in range(len(ratio)):
  for j in range(len(anchor_scales)):
    h = sub_sample * anchor_scales[j] * np.sqrt(ratio[i])
    w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratio[i])
    index = i * len(anchor_scales) + j
    anchor_base[index, 0] = ctr_y - h / 2.
    anchor_base[index, 1] = ctr_x - w / 2.
    anchor_base[index, 2] = ctr_y + h / 2.
    anchor_base[index, 3] = ctr_x + w / 2.
print(anchor_base)
#These are the anchor locations at the first feature map pixel, we have to now generate these anchors at all the locations of feature map

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


8.0 8.0
[[ -37.254833  -82.50967    53.254833   98.50967 ]
 [ -82.50967  -173.01933    98.50967   189.01933 ]
 [-173.01933  -354.03867   189.01933   370.03867 ]
 [ -56.        -56.         72.         72.      ]
 [-120.       -120.        136.        136.      ]
 [-248.       -248.        264.        264.      ]
 [ -82.50967   -37.254833   98.50967    53.254833]
 [-173.01933   -82.50967   189.01933    98.50967 ]
 [-354.03867  -173.01933   370.03867   189.01933 ]]



Generate Anchor at all the feature map location.

In [ ]:
# centers for each feature map
fe_size = (800//16)
ctr_x = np.arange(16, (fe_size+1) * 16, 16)
ctr_y = np.arange(16, (fe_size+1) * 16, 16)

# # Looping through the ctr_x and ctr_y will give us the centers at each and every location. The sudo code is as a below

feature_map_size = (50, 50)
ctr = np.zeros((*feature_map_size, 2))
for idx, y in enumerate(ctr_y):
    ctr[idx, :, 0] = y
    ctr[idx, :, 1] = ctr_x
print(ctr.shape)

anchors = np.zeros((*feature_map_size, 9, 4))

for idx_y in range(feature_map_size[0]):
    for idx_x in range(feature_map_size[1]):
        anchors[idx_y, idx_x] = (ctr[idx_y, idx_x] + anchor_base.reshape(-1, 2, 2)).reshape(-1, 4)
print(anchors.shape)

In [ ]:
# Now at each center we need to generate the anchor boxes.
anchors = anchors.reshape(-1, 4)
print(anchors.shape)

(22500, 4)


 Assign the labels and location of objects (with respect to the anchor) to each and every anchor.

In [ ]:
bbox = np.asarray([[20, 30, 400, 500], [300, 400, 500, 600]], dtype=np.float32) # [y1, x1, y2, x2] format
labels = np.asarray([6, 8], dtype=np.int8)

index_inside = np.where(
        (anchors[:, 0] >= 0) &
        (anchors[:, 1] >= 0) &
        (anchors[:, 2] <= 800) &
        (anchors[:, 3] <= 800)
    )[0]
print(index_inside.shape)

In [ ]:
valid_anchors = anchors[index_inside]

creating an empty label array with inside_index shape and fill with -1. Default is set to (d)

In [ ]:
label = np.empty((len(index_inside), ), dtype=np.int32)
label.fill(-1)
print(label.shape)

(8940,)


In [ ]:
# create an array with valid anchor boxes
valid_anchor_boxes = anchors[index_inside]
print(valid_anchor_boxes.shape)

(8940, 4)


In [ ]:
""" For each valid anchor box calculate the iou with each ground truth object. 
Since we have 8940 anchor boxes and 2 ground truth objects, we should get an array with (8490, 2) as the output. 
The sudo code for calculating iou between two boxes will be """

ious = np.zeros((len(valid_anchors), len(bbox)), dtype=np.float32)

for index_v, valid_anchor in enumerate(valid_anchors):
    y1_v, x1_v, y2_v, x2_v = valid_anchor
    anchor_area = (y2_v - y1_v) * (x2_v - x1_v)
    
    for index_t, bbox_t in enumerate(bbox):
        y1_t, x1_t, y2_t, x2_t = bbox_t
        bbox_area = (y2_t - y1_t) * (x2_t - x1_t)
        
        max_y1 = max([y1_v, y1_t])
        max_x1 = max([x1_v, x1_t])
        min_y2 = min([y2_v, y2_t])
        min_x2 = min([x2_v, x2_t])
        
        if max_x1 < min_x2 and max_y1 < min_y2:
            inter_area = (min_y2 - max_y1) * (min_x2 - max_x1)
            iou = inter_area / (anchor_area + bbox_area - inter_area)
        else:
            iou = 0
        
        ious[index_v, index_t] = iou
print(ious.shape)

(8940, 2)


In [ ]:
pos_iou_thres = 0.7
neg_iou_thred = 0.3

anchor_max_iou = np.amax(ious, axis=1)
pos_iou_anchor_label = np.where(anchor_max_iou >= pos_iou_thres)[0]
neg_iou_anchor_label = np.where(anchor_max_iou < neg_iou_thred)[0]
label[pos_iou_anchor_label] = 1
label[neg_iou_anchor_label] = 0

 Assigning labels to the anchors with highest iou with box

In [ ]:
gt_max_iou = np.amax(ious, axis=0)
gt_max_iou_anchor_label = np.where(ious == gt_max_iou)[0]
print(gt_max_iou_anchor_label)
label[gt_max_iou_anchor_label] = 1

[2262 2269 4096 4104 4112 4120 4370 4378 4386 4394 4644 4652 4660 4668
 4918 4926 4934 4942]


In [ ]:
gt_argmax_ious = ious.argmax(axis=0)
print(gt_argmax_ious)
gt_max_ious = ious[gt_argmax_ious, np.arange(ious.shape[1])]
print(gt_max_ious)

[2262 4096]
[0.68130493 0.61035156]


In [ ]:
argmax_ious = ious.argmax(axis=1)
print(argmax_ious.shape)
print(argmax_ious)
max_ious = ious[np.arange(len(index_inside)), argmax_ious]
print(max_ious)

(8940,)
[0 0 0 ... 0 0 0]
[0.07360116 0.08134178 0.08159018 ... 0.         0.         0.        ]


In [ ]:
# Find the anchor_boxes which have this max_ious (gt_max_ious)
gt_argmax_ious = np.where(ious == gt_max_ious)[0]
print(gt_argmax_ious)

[2262 2269 4096 4104 4112 4120 4370 4378 4386 4394 4644 4652 4660 4668
 4918 4926 4934 4942]


In [ ]:
pos_iou_threshold  = 0.7
neg_iou_threshold = 0.3
# Assign negitive label (0) to all the anchor boxes which have max_iou less than negitive threshold [c]
label[max_ious < neg_iou_threshold] = 0
# Assign positive label (1) to all the anchor boxes which have highest IoU overlap with a ground-truth box [a]
label[gt_argmax_ious] = 1
# Assign positive label (1) to all the anchor boxes which have max_iou greater than positive threshold [b]
label[max_ious >= pos_iou_threshold] = 1

## Training RPN

In [ ]:
pos_ratio = 0.5
n_sample = 256
# total positive samples
n_pos = pos_ratio * n_sample
n_pos

128.0

In [ ]:
""" Now we need to randomly sample n_pos samples from the positive labels and ignore (-1) the remaining ones. 
In some cases we get less than n_pos samples, in that we will randomly sample (n_sample — n_pos) negitive samples (0) 
and assign ignore label to the remaining anchor boxes. This is done using the following code."""
# Positive samples
pos_index = np.where(label == 1)[0]
if len(pos_index) > n_pos:
    disable_index = np.random.choice(pos_index, size=(len(pos_index) - n_pos), replace=False)
    label[disable_index] = -1

In [ ]:
# Negative samples
n_neg = n_sample * np.sum(label == 1)
neg_index = np.where(label == 0)[0]
if len(neg_index) > n_neg:
    disable_index = np.random.choice(neg_index, size=(len(neg_index) - n_neg), replace = False)
    label[disable_index] = -1

#### Assigning locations to anchor boxes

In [ ]:
# For each anchor box, find the groundtruth object which has max_iou
max_iou_bbox = bbox[argmax_ious]
print(max_iou_bbox)

[[ 20.  30. 400. 500.]
 [ 20.  30. 400. 500.]
 [ 20.  30. 400. 500.]
 ...
 [ 20.  30. 400. 500.]
 [ 20.  30. 400. 500.]
 [ 20.  30. 400. 500.]]


In [ ]:


height = valid_anchors[:, 2] - valid_anchors[:, 0]
width = valid_anchors[:, 3] - valid_anchors[:, 1]
ctr_y = valid_anchors[:, 0] + height*0.5
ctr_x = valid_anchors[:, 1] + width*0.5

base_height = max_iou_bbox[:, 2] - max_iou_bbox[:, 0]
base_width = max_iou_bbox[:, 3] - max_iou_bbox[:, 1]
base_ctr_y = max_iou_bbox[:, 0] + base_height*0.5
base_ctr_x = max_iou_bbox[:, 1] + base_width*0.5

In [ ]:
# Use the above formulas to find the loc

eps = np.finfo(height.dtype).eps
height = np.maximum(height, eps)
width = np.maximum(width, eps)

dy = (base_ctr_y - ctr_y) / height
dx = (base_ctr_x - ctr_x) / width
dh = np.log(base_height / height)
dw = np.log(base_width / width)

anchor_locs = np.vstack((dy, dx, dh, dw)).transpose()
print(anchor_locs)

[[ 1.70147572  0.88940776  1.4347146   0.95412886]
 [ 1.70147572  0.80101941  1.4347146   0.95412886]
 [ 1.70147572  0.71263107  1.4347146   0.95412886]
 ...
 [-5.89992231 -2.20418446  1.4347146   0.95412886]
 [-5.89992231 -2.29257281  1.4347146   0.95412886]
 [-5.89992231 -2.38096116  1.4347146   0.95412886]]


In [ ]:
# Lets map them to the original anchors using the inside_index variable. Fill the unvalid anchor boxes labels with -1 (ignore) and locations with 0.

anchor_labels = np.empty((len(anchors),), dtype=np.int32)
anchor_labels.fill(-1)
anchor_labels[index_inside] = label

In [ ]:
anchor_locations = np.empty((len(anchors), 4), dtype=np.float32)
anchor_locations.fill(0)
anchor_locations[index_inside] = anchor_locs

### Region Proposal Network

* A box regrression layer
* A box classification layer

In [ ]:
import torch.nn as nn

mid_channels = 512
in_channels = 512 # depends on the output feature map. in vgg 16 it is equal to 512
n_anchor = 9 # Number of anchors at each location

conv1 = nn.Conv2d(in_channels, mid_channels, 3, 1, 1)
reg_layer = nn.Conv2d(mid_channels, n_anchor *4, 1, 1, 0)
cls_layer = nn.Conv2d(mid_channels, n_anchor *2, 1, 1, 0) ## I will be going to use softmax here. you can equally use sigmoid if u replace 2 with 1.

In [ ]:
# The paper tells that they initialized these layers with zero mean and 0.01 standard deviation for weights and zeros for base. Lets do that

# conv sliding layer
conv1.weight.data.normal_(0, 0.01)
conv1.bias.data.zero_()
# Regression layer
reg_layer.weight.data.normal_(0, 0.01)
reg_layer.bias.data.zero_()
# classification layer
cls_layer.weight.data.normal_(0, 0.01)
cls_layer.bias.data.zero_()

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
# Now the outputs we got in the feature extraction state should be sent to this network to predict locations of objects 
# with repect to the anchor and the objectness score assoiciated with it.

x = conv1(out_map) # out_map is obtained in section 1
pred_anchor_locs = reg_layer(x)
pred_cls_scores = cls_layer(x)
print(pred_cls_scores.shape, pred_anchor_locs.shape)

torch.Size([1, 18, 50, 50]) torch.Size([1, 36, 50, 50])


In [ ]:
# Reformat the output

pred_anchor_locs = pred_anchor_locs.permute(0, 2, 3, 1).contiguous().view(1, -1, 4)
print(pred_anchor_locs.shape)

pred_cls_scores = pred_cls_scores.permute(0, 2, 3, 1).contiguous()
print(pred_cls_scores.shape)

objectness_scores = pred_cls_scores.view(1, 50, 50, 9, 2)[:, :, :, :, 1].contiguous().view(1, -1)
print(objectness_scores.shape)

pred_cls_scores  = pred_cls_scores.view(1, -1, 2)
print(pred_cls_scores.shape)

torch.Size([1, 22500, 4])
torch.Size([1, 50, 50, 18])
torch.Size([1, 22500])
torch.Size([1, 22500, 2])


Generating proposals to feed Fast R-CNN network
* Weather training_mode or testing mode
* nms_thresh
* n_train_pre_nms — number of bboxes before nms during training
* n_train_post_nms — number of bboxes after nms during training
* n_test_pre_nms — number of bboxes before nms during testing
* n_test_post_nms — number of bboxes after nms during testing
* min_size — minimum height of the object required to create a proposal.


In [ ]:
nms_thresh = 0.7
n_train_pre_nms = 12000
n_train_post_nms = 2000
n_test_pre_nms = 6000
n_test_post_nms = 300
min_size = 16

We need to do the following things to generate region of interest proposals to the network.

* convert the loc predictions from the rpn network to bbox [y1, x1, y2, x2] format.
* clip the predicted boxes to the image
* Remove predicted boxes with either height or width < threshold (min_size).
* Sort all (proposal, score) pairs by score from highest to lowest.
* Take top pre_nms_topN (e.g. 12000 while training and 300 while testing).
* Apply nms threshold > 0.7
* Take top pos_nms_topN (e.g. 2000 while training and 300 while testing)

In [ ]:
# Step 1 - convert the loc predictions from the rpn net to [y1, x1, y2, x2] format

anc_height = anchors[:, 2] - anchors[:, 0]
anc_width = anchors[:, 3] - anchors[:, 1]
anc_ctr_y = anchors[:, 0] + anc_height*0.5
anc_ctr_x = anchors[:, 1] + anc_width*0.5

pred_anchor_locs_np = pred_anchor_locs[0].data.numpy()
objectness_score_np = objectness_scores[0].data.numpy()

dy = pred_anchor_locs_np[:, 0]
dx = pred_anchor_locs_np[:, 1]
dh = pred_anchor_locs_np[:, 2]
dw = pred_anchor_locs_np[:, 3]

ctr_y = dy * anc_height + anc_ctr_y
ctr_x = dx * anc_width + anc_ctr_x
h = np.exp(dh) * anc_height
w = np.exp(dw) * anc_width

# generate roi(region of interest) matrix
roi = np.zeros(pred_anchor_locs_np.shape, dtype=anchors.dtype)

roi[:, 0] = ctr_y - 0.5*h
roi[:, 1] = ctr_x - 0.5*w
roi[:, 2] = ctr_y + 0.5*h
roi[:, 3] = ctr_x + 0.5*w

roi

array([[ -21.27012452,  -66.88055466,   68.8818831 ,  110.64189271],
       [ -72.50248627, -146.78686319,  113.13822353,  220.02960702],
       [-137.64253878, -342.39584466,  219.0742897 ,  400.2782568 ],
       ...,
       [ 718.2551547 ,  761.79419658,  899.79592704,  852.36371289],
       [ 635.18572238,  715.72762999,  999.52136304,  894.44276975],
       [ 459.01098488,  630.99857736, 1167.25035912,  991.39776447]])

In [ ]:
# Step 2: clip the predicted boxes to the image

img_size = (800, 800) #Image size
roi[:, slice(0, 4, 2)] = np.clip(roi[:, slice(0, 4, 2)], 0, img_size[0])
roi[:, slice(1, 4, 2)] = np.clip(roi[:, slice(1, 4, 2)], 0, img_size[1])
print(roi)

[[  0.           0.          68.8818831  110.64189271]
 [  0.           0.         113.13822353 220.02960702]
 [  0.           0.         219.0742897  400.2782568 ]
 ...
 [718.2551547  761.79419658 800.         800.        ]
 [635.18572238 715.72762999 800.         800.        ]
 [459.01098488 630.99857736 800.         800.        ]]


In [ ]:
# Step 3: Remove predicted boxes with either height or width < threshold

keep = np.where((h >= min_size) & (w >= min_size))[0]
roi = roi[keep]
score = objectness_score_np[keep]

print(score.shape)

(22500,)


In [ ]:
# Step 4: sort all pairs by score from highest to lowest

order = score.ravel().argsort()[::-1]
print(order)

[  889   457 22487 ...    12   895 22040]


In [ ]:
# Step 5: Take top pre_nms_topN

order = order[:n_train_pre_nms]
roi = roi[order]
score = score[order]

print(roi.shape)
print(score.shape)

(12000, 4)
(12000,)


In [ ]:
# Step 7: Take top pos_nms_topN

y1, x1, y2, x2 = roi.T

areas = (y2-y1+1) * (x2-x1+1)

order = score.argsort()[::-1]

keep = []

while order.size > 0:
    i = order[0]
    keep.append(i)

    yy1 = np.maximum(y1[i], y1[order[1:]])
    xx1 = np.maximum(x1[i], x1[order[1:]])
    yy2 = np.maximum(y2[i], y2[order[1:]])
    xx2 = np.maximum(y2[i], y2[order[1:]])
    
    h = np.maximum(0.0, yy2-yy1)
    w = np.maximum(0.0, xx2-xx1)
    
    inter = h*w
    iou = inter / (areas[i] + areas[order[1:]] - inter)
    
    # IOU threshold
    inds = np.where(iou <= nms_thresh)[0]

    order = order[inds+1]

keep = keep[:n_train_post_nms]
roi = roi[keep]
print(roi.shape)

(1405, 4)


### Proposal targets
* n_sample: Number of samples to sample from roi, The default value is 128.
* pos_ratio: the number of positive examples out of the n_samples. The default values is 0.25.
* pos_iou_thesh: The minimum overlap of region proposal with any groundtruth object to consider it as positive label.
* [neg_iou_threshold_lo, neg_iou_threshold_hi] : [0.0, 0.5], The overlap value bounding required to consider a region proposal as negitive [background object].

In [ ]:
n_sample = 128
pos_ratio = 0.25
pos_iou_thresh = 0.5
neg_iou_thresh_hi = 0.5
neg_iou_thresh_lo = 0.0

In [ ]:
ious = np.empty((len(roi), 2), dtype = np.float32)
ious.fill(0)

for index, box in enumerate(bbox):
    y1, x1, y2, x2 = box
    
    inter_y1 = np.maximum(y1, roi[:, 0])
    inter_x1 = np.maximum(x1, roi[:, 1])
    inter_y2 = np.minimum(y2, roi[:, 2])
    inter_x2 = np.minimum(x2, roi[:, 3])
    
    h = np.maximum(0.0, inter_y2-inter_y1)
    w = np.maximum(0.0, inter_x2-inter_x1)
    
    eps = np.finfo(np.float32).eps
    iou = h*w / ((y2-y1)*(x2-x1)+(roi[:, 2]-roi[:, 0])*(roi[:, 3]-roi[:, 1]) - h*w + eps)
        
    ious[:, index] = iou
print(ious.shape)

(1405, 2)


#### Find out which ground truth has high IoU for each region proposal, Also find the maximum IoU

In [ ]:
gt_assignment = ious.argmax(axis=1)
max_iou = ious.max(axis=1)
print(gt_assignment)
print(max_iou)

[0 0 0 ... 1 1 0]
[0.         0.08874738 0.         ... 0.03842504 0.00312049 0.        ]


#### Assign the labels to each proposal

In [ ]:
gt_roi_label = labels[gt_assignment]

In [ ]:
gt_roi_label

array([6, 6, 6, ..., 8, 8, 6], dtype=int8)

#### Select the foreground rois as per the pos_iou_thesh. We also want only n_sample x pos_ratio (128 x 0.25 = 32) foreground samples. So incase if we get less than 32 positive samples we will leave it as it is, Incase if we get more than 32 foreground samples, we will sample 32 samples from the positive samples. This is done using the following code.

In [ ]:
pos_index = np.where(max_iou>=pos_iou_thresh)[0]
pos_roi_per_this_image = n_sample * pos_ratio
pos_roi_per_this_image = int(min(pos_roi_per_this_image, pos_index.size))
if pos_index.size > 0:
    pos_index = np.random.choice(pos_index, size=pos_roi_per_this_image, replace=False)
print(pos_roi_per_this_image)
print(pos_index.shape)

0
(0,)


In [ ]:
neg_index = np.where((max_iou < neg_iou_thresh_hi) & (max_iou > neg_iou_thresh_lo))[0]
neg_roi_per_this_image = n_sample - pos_roi_per_this_image

if neg_index.size > 0:
    neg_index = np.random.choice(neg_index, size=neg_roi_per_this_image, replace=False)
print(neg_roi_per_this_image)
print(neg_index.shape)

128
(128,)


#### Now we gather positve samples index and negitive samples index, their respective labels and region proposals

In [ ]:
keep_index = np.append(pos_index, neg_index)
print(len(keep_index))
gt_roi_labels = gt_roi_label[keep_index]
gt_roi_labels[pos_roi_per_this_image:] = 0
sample_roi = roi[keep_index]
print(sample_roi.shape)

128
(128, 4)


#### Pick the ground truth objects for these sample_roi and later parameterize as we have done while assigning locations to anchor boxes in section 2.


In [ ]:
bbox_for_sampled_roi = bbox[gt_assignment[keep_index]]

height = sample_roi[:, 2] - sample_roi[:, 0]
width = sample_roi[:, 3] - sample_roi[:, 2]
ctr_y = sample_roi[:, 0] + height*0.5
ctr_x = sample_roi[:, 1] + width*0.5

base_height = bbox_for_sampled_roi[:, 2] - bbox_for_sampled_roi[:, 0]
base_width = bbox_for_sampled_roi[:, 3] - bbox_for_sampled_roi[:, 1]
base_ctr_y = bbox_for_sampled_roi[:, 0] + base_height*0.5
base_ctr_x = bbox_for_sampled_roi[:, 1] + base_width*0.5

In [ ]:
eps = np.finfo(height.dtype).eps
height = np.maximum(height, eps)
width = np.maximum(width, eps)
dy = (base_ctr_y - ctr_y) / height
dx = (base_ctr_x - ctr_x) / width
dh = np.log(base_height / height)
dw = np.log(base_width / width)
gt_roi_locs = np.vstack((dy, dx, dh, dw)).transpose()
print(gt_roi_locs)

[[ 1.15596829e+00 -6.76017269e-01  1.09744963e+00  5.22743420e-01]
 [-5.96004290e-01 -2.31079156e-01  1.43975157e+00  1.46713827e+00]
 [ 1.72584194e+00 -2.66197978e-01  1.44478515e+00  5.03469427e-01]
 [-7.73673655e-01 -1.84500009e-01  1.43975157e+00  1.62687985e+00]
 [ 2.87473563e-01 -4.47492763e-01 -9.32072660e-01 -3.78595463e-01]
 [ 4.34512266e-01  4.72952061e-01  5.25333195e-01  1.44607928e+00]
 [ 1.00303038e+00 -5.62775271e-01  1.43976691e+00  4.47664869e-01]
 [ 1.99427030e-01 -7.13505592e-01 -1.05064099e+00 -1.31481296e-01]
 [ 1.35941615e+00 -2.58619791e-01  1.44004264e+00  6.22075538e-01]
 [ 1.97935758e-01 -6.94779968e-01 -9.52752021e-01 -1.96222114e-01]
 [ 8.36776071e-01 -7.16570344e-01  1.09457228e+00  4.83998647e-01]
 [ 1.08850809e+00 -6.53860496e-01  1.09845752e+00  4.28604141e-01]
 [ 4.70011899e-01  1.63539984e-01  1.43975157e+00  1.20841761e+00]
 [ 2.08029030e-01  2.98350673e-01  1.09465435e+00  1.74957623e+00]
 [ 2.40807475e-01 -6.25629166e-01 -9.93161686e-01 -2.62551342e

### Fast R-CNN



* A fixed-size feature map obtained from a deep convolutional network with several convolutions and max-pooling layers
* An Nx5 matrix of representing a list of regions of interest, where N is the number of RoIs. The first column represents the image index and the remaining four are the co-ordinates of the top left and bottom right corners of the region.

What does the RoI pooling actually do? For every region of interest from the input list, it takes a section of the input feature map that corresponds to it and scales it to some pre-defined size (e.g., 7×7). The scaling is done by:

* Dividing the region proposal into equal-sized sections (the number of which is the same as the dimension of the output)
* Finding the largest value in each section
Copying these max values to the output buffer



In [ ]:
rois = torch.from_numpy(sample_roi).float()
# roi_indices is 0 because there is only one image
roi_indices = 0 * np.ones((len(rois), 1), dtype=np.int32)
roi_indices = torch.from_numpy(roi_indices).float()
print(rois.shape, roi_indices.shape)

torch.Size([128, 4]) torch.Size([128, 1])


In [ ]:
indices_and_rois = torch.cat([roi_indices, rois], dim=1)
print(indices_and_rois.shape)

torch.Size([128, 5])


In [ ]:
# Now we need to pass this array to the roi_pooling layer. We will briefly discuss the workings of it here. The sudo code is as follows

# - Multiply the dimensions of rois with the sub_sampling ratio (16 in this case)
# - Empty output Tensor
# - Take each roi
#     - subset the feature map based on the roi dimension
#     - Apply AdaptiveMaxPool2d to this subset Tensor.
#     - Add the outputs to the output Tensor
# - Empty output Tensor goes to the network

We will define the size to be 7 x 7 and define adaptive_max_pool

In [ ]:
size = (7, 7)
adaptive_max_pool = nn.AdaptiveAvgPool2d(size)

output = []
rois = indices_and_rois.data.float()
rois[:, 1:].mul_(1/16.0)
rois = rois.long()
num_rois = rois.size(0)
print(f'num_ris = {num_rois}')

for i in range(num_rois):
    roi = rois[i]
    im_idx = roi[0]
    im = sample_output[..., roi[1]:roi[3]+1, roi[2]:roi[4]+1]
    output.append(adaptive_max_pool(im))

output = torch.cat(output, 0)
print(output.size())

num_ris = 128
torch.Size([128, 512, 7, 7])


In [ ]:
# Reshape the tensor so that we can pass it through the feed forward layer.
k = output.view(output.size(0), -1)
print(k.size())

torch.Size([128, 25088])


This will be the input to a classifier layer, which will further brach out to a classification head and regression head as shown in the diagram below. lets define the network

In [ ]:
roi_head_classifier = nn.Sequential(nn.Linear(25088, 4096),
                                    nn.Linear(4096, 4096))

cls_loc = nn.Linear(4096, 21*4) # (VOC 20 classes + 1 background. Each will have 4 co-ordinates)

cls_loc.weight.data.normal_(0, 0.01)
cls_loc.bias.data.zero_()

score = nn.Linear(4096, 21) # (VOC 20 classes + 1 background)

In [ ]:
k = roi_head_classifier(k)
roi_cls_loc = cls_loc(k)
roi_cls_score = score(k)
print(roi_cls_loc.shape, roi_cls_score.shape)

torch.Size([128, 84]) torch.Size([128, 21])


### Loss functions

We have two networks, RPN and Fast-RCNN, which further have two outputs each (Regression head, and classification head)

In [ ]:
print(pred_anchor_locs.shape)
print(pred_cls_scores.shape)
print(anchor_locations.shape)
print(anchor_labels.shape)

torch.Size([1, 22500, 4])
torch.Size([1, 22500, 2])
(22500, 4)
(22500,)


#### We will re-arrange a bit so that the inputs and outputs align

In [ ]:
rpn_loc = pred_anchor_locs[0]
rpn_score = pred_cls_scores[0]

gt_rpn_loc = torch.from_numpy(anchor_locations)
gt_rpn_score = torch.from_numpy(anchor_labels)

In [ ]:
import torch.nn.functional as F
# F.cross_entropy already average the loss for each batch
rpn_cls_loss = F.cross_entropy(rpn_score, gt_rpn_score.long(), ignore_index=-1)
print(rpn_cls_loss)

tensor(0.6962, grad_fn=<NllLossBackward0>)


In [ ]:
pos = gt_rpn_score > 0
mask = pos.unsqueeze(1).expand_as(rpn_loc)
print(mask.shape)

torch.Size([22500, 4])


In [ ]:
# Now take those bounding boxes which have positve labels
mask_loc_preds = rpn_loc[mask].view(-1, 4)
mask_loc_targets = gt_rpn_loc[mask].view(-1, 4)
print(mask_loc_preds.shape, mask_loc_preds.shape)

torch.Size([18, 4]) torch.Size([18, 4])


In [ ]:
# The regression loss is applied as following
x = torch.abs(mask_loc_targets - mask_loc_preds)
rpn_loc_loss = ((x < 1).float() * 0.5 * x**2) + ((x >= 1).float() * (x-0.5))
print(rpn_loc_loss.sum())

tensor(1.1951, grad_fn=<SumBackward0>)


In [ ]:
rpn_lambda = 10.
N_reg = (gt_rpn_score >0).float().sum()
rpn_loc_loss = rpn_loc_loss.sum() / N_reg
rpn_loss = rpn_cls_loss + (rpn_lambda * rpn_loc_loss)
print(rpn_loss)

tensor(1.3601, grad_fn=<AddBackward0>)


#### predicted

In [ ]:
print(roi_cls_loc.shape)
print(roi_cls_score.shape)

torch.Size([128, 84])
torch.Size([128, 21])


#### Actual

In [ ]:
print(gt_roi_locs.shape)
print(gt_roi_labels.shape)

(128, 4)
(128,)


#### Converting ground truth to torch variable

In [ ]:
gt_roi_loc = torch.from_numpy(gt_roi_locs)
gt_roi_label = torch.from_numpy(np.float32(gt_roi_labels)).long()
print(gt_roi_loc.shape, gt_roi_label.shape)

torch.Size([128, 4]) torch.Size([128])


#### Classification loss

In [ ]:
roi_cls_loss = F.cross_entropy(roi_cls_score, gt_roi_label, ignore_index=-1)
print(roi_cls_loss)

tensor(3.0446, grad_fn=<NllLossBackward0>)


#### calculating regression loss 

In [ ]:
mask = gt_roi_label>0
mask_loc_preds = roi_loc[mask].view(-1, 4)
mask_loc_targets = gt_roi_loc[mask].view(-1, 4)

print(mask_loc_preds.shape, mask_loc_targets.shape)

x = torch.abs(mask_loc_preds - mask_loc_targets)
roi_loc_loss = (x<1).float()*0.5*x**2 + (x>=1).float()*(x-0.5)
roi_loc_loss = roi_loc_loss
print(roi_loc_loss.sum())

torch.Size([0, 4]) torch.Size([0, 4])
tensor(0., dtype=torch.float64, grad_fn=<SumBackward0>)


In [ ]:
roi_lambda = 10.
N_reg = (gt_roi_label>0).float().sum()
roi_loc_loss = roi_loc_loss.sum() / N_reg
roi_loss = roi_cls_loss + roi_lambda * roi_loc_loss
print(roi_loss)

tensor(nan, dtype=torch.float64, grad_fn=<AddBackward0>)


In [ ]:
total_loss = rpn_loss + roi_loss
print(total_loss)

tensor(nan, dtype=torch.float64, grad_fn=<AddBackward0>)
